In [1]:
from tensorflow.keras.models import load_model
from kapre.time_frequency import STFT, Magnitude, ApplyFilterbank, MagnitudeToDecibel
from sklearn.preprocessing import LabelEncoder
import argparse
import tensorflow as tf

In [2]:
import matplotlib.pyplot as plt
from scipy.io import wavfile
import argparse
import os
from glob import glob
import numpy as np
import pandas as pd
from librosa.core import resample, to_mono
from tqdm import tqdm
import wavio
import soundfile as sf

In [3]:
args={'model_fn':'models/cnn_pretrained.h5',
      'pred_fn':'y_pred',
     'dt':1.0,
     'threshold':20,
     'sr':16000,
      'src_dir':'pred_final'
     }

In [4]:
#args={'model_fn':'models/cnn_pretrained.h5',
     # 'pred_fn':'y_pred',
     #'dt':1.0,
     #'threshold':20,
     #'sr':16000,
     # 'src_dir':'pred-final2'
     #}

In [ ]:
def enveloppe_mean(y, rate, threshold):
    mask=[]
    y = np.abs(y)
    y_mean = pd.Series(y).rolling(window=int(rate/20), min_periods=1, center=True).mean()
    mask = [True if m > threshold else False for m in y_mean]
    return mask

In [5]:
def enveloppe_max(y, rate, threshold):
    mask=[]
    y = np.abs(y)
    y_mean = pd.Series(y).rolling(window=int(rate/20), min_periods=1, center=True).max()
    mask = [True if m > threshold else False for m in y_mean]
    return mask, y_mean

In [6]:
def downsample_mono_v2(path, sr):
    wavs = []
    obj = wavio.read(path)
    wav1 = obj.data.astype(np.float32, order='F')
    rate = obj.rate
    #print(wav1.shape[0])
    # nouveaux audio de 03s
    for i in range(0, len(wav1), rate*3):
        #print("mono_v2 " + str(i))
        wav = []
        for j in range(len(wav1)-i):
            wav.append(wav1[i+j])
        wav = np.array(wav, dtype=np.float32)
        try:
            channel = wav.shape[1]
            if channel == 2:
                wav = to_mono(wav.T)
            elif channel == 1:
                wav = to_mono(wav.reshape(-1))
        except IndexError:
            wav = to_mono(wav.reshape(-1))
            pass
        except Exception as exc:
            raise exc
        wav = resample(wav, rate, sr)
        wav = wav.astype(np.int16)
        wavs.append(wav)
    return sr, wavs

In [7]:
def downsample_mono(path, sr):
    obj = wavio.read(path)
    wav = obj.data.astype(np.float32, order='F')
    rate = obj.rate
    wav = wav.reshape(-1) if wav.ndim == 1 else to_mono(wav.T)
    wav = resample(wav, rate, sr)
    wav = wav.astype(np.int16)
    return sr, wav

In [8]:
def make_prediction(args):

    model = load_model(args['model_fn'],
                       custom_objects={'STFT': STFT,
                                       'Magnitude': Magnitude,
                                       'ApplyFilterbank': ApplyFilterbank,
                                       'MagnitudeToDecibel': MagnitudeToDecibel})
    wav_paths = glob('{}/**'.format(args['src_dir']), recursive=True)
    wav_paths = sorted([x.replace(os.sep, '/')
                       for x in wav_paths if '.wav' in x])
    classes = sorted(os.listdir(args['src_dir']))
    labels = [os.path.split(x)[0].split('/')[-1] for x in wav_paths]
    le = LabelEncoder()
    y_true = le.fit_transform(labels)
    results = []

    for z, wav_fn in tqdm(enumerate(wav_paths), total=len(wav_paths)):
        #print(wav_fn)
        rate, wavs = downsample_mono_v2(wav_fn, args['sr'])

        loc_results = []
        y_means = []
        for wav in wavs:
            #print("entered first split")
            mask, env = enveloppe_max(wav, rate, args['threshold'])
            clean_wav = wav[mask]
            step = int(args['sr']*args['dt'])
            batch = []

            for i in range(0, clean_wav.shape[0], step):
                #print("entered second split")
                sample = clean_wav[i:i+step]
                sample = sample.reshape(-1, 1)
                if sample.shape[0] < step:
                    tmp = np.zeros(shape=(step, 1), dtype=np.float32)
                    tmp[:sample.shape[0], :] = sample.flatten().reshape(-1, 1)
                    sample = tmp
                batch.append(sample)
            X_batch = np.array(batch, dtype=np.float32)
            y_pred = model.predict(X_batch)
            y_mean = np.mean(y_pred, axis=0)
            # rajoute un if ici, si un seuil est dépassé alors ..., sinon argmax
            y_pred = np.argmax(y_mean)
            real_class = os.path.dirname(wav_fn).split('/')[-1]
            loc_results.append(y_pred)
        string_results = "".join(map(str, loc_results))
        if "000" in string_results:
            print('Actual class: {}, Predicted class : {}'.format(
                real_class, classes[0]))
        else:
            rate, wav_solo = downsample_mono(wav_fn, args['sr'])
            mask, env = enveloppe_max(wav_solo, rate, threshold=args['threshold'])
            clean_wav = wav_solo[mask]
            step = int(args['sr']*args['dt'])
            batch = []

            for i in range(0, clean_wav.shape[0], step):
                sample = clean_wav[i:i+step]
                sample = sample.reshape(-1, 1)
                if sample.shape[0] < step:
                    tmp = np.zeros(shape=(step, 1), dtype=np.float32)
                    tmp[:sample.shape[0], :] = sample.flatten().reshape(-1, 1)
                    sample = tmp
                batch.append(sample)
            X_batch = np.array(batch, dtype=np.float32)
            y_pred = model.predict(X_batch)
            y_mean = np.mean(y_pred, axis=0)
            # rajoute un if ici, si un seuil est dépassé alors ..., sinon argmax
            y_pred = np.argmax(y_mean)
            print('Actual class: {}, Predicted class : {}'.format(
                real_class, classes[y_pred]))

    np.save(os.path.join('logs', args['pred_fn']), np.array(results))


In [23]:
#ancienne approche
make_prediction(args)

  1%|█                                                                                  | 1/82 [00:01<02:12,  1.64s/it]

Actual class: both, Predicted class : both


  2%|██                                                                                 | 2/82 [00:01<01:38,  1.23s/it]

Actual class: both, Predicted class : both


  4%|███                                                                                | 3/82 [00:02<01:26,  1.09s/it]

...

Actual class: plane, Predicted class : plane


 99%|██████████████████████████████████████████████████████████████████████████████ | 81/82 [1:50:37<02:01, 121.71s/it]

Actual class: plane, Predicted class : plane


100%|████████████████████████████████████████████████████████████████████████████████| 82/82 [1:52:52<00:00, 82.60s/it]

Actual class: plane, Predicted class : plane


In [8]:
#pretrained sur 1ère data de validation (la data a été découpée en deux à cause d'un problème de mémoire)
make_prediction(args)

  2%|█▍                                                                                 | 1/59 [00:01<01:35,  1.65s/it]

Actual class: both, Predicted class : both


  3%|██▊                                                                                | 2/59 [00:01<01:09,  1.23s/it]

Actual class: both, Predicted class : both


  5%|████▏                                                                              | 3/59 [00:02<01:00,  1.07s/it]

...

Actual class: plane, Predicted class : plane


 98%|███████████████████████████████████████████████████████████████████████████████▋ | 58/59 [57:19<02:02, 122.60s/it]

Actual class: plane, Predicted class : plane


100%|██████████████████████████████████████████████████████████████████████████████████| 59/59 [59:18<00:00, 60.31s/it]

Actual class: plane, Predicted class : plane


In [9]:
#pretrained sur 2ème data de validation (la data a été découpée en deux à cause d'un problème de mémoire)
make_prediction(args)

  4%|███▍                                                                            | 1/23 [04:15<1:33:50, 255.92s/it]

Actual class: both, Predicted class : both


  9%|██████▉                                                                         | 2/23 [08:33<1:29:45, 256.43s/it]

Actual class: both, Predicted class : both


 13%|██████████▍                                                                     | 3/23 [10:28<1:11:17, 213.88s/it]

...

Actual class: plane, Predicted class : plane


 96%|█████████████████████████████████████████████████████████████████████████████▍   | 22/23 [48:05<01:58, 118.44s/it]

Actual class: plane, Predicted class : plane


100%|█████████████████████████████████████████████████████████████████████████████████| 23/23 [50:01<00:00, 130.52s/it]

Actual class: plane, Predicted class : plane
